In [1]:
import sys
sys.path.append("/home/brunoperdigao/repos/Plan-Classification-Topologic-DGL")

In [2]:
# Import Libraries
import time
import sklearn
import pandas as pd
from topologicpy.DGL import DGL
from topologicpy.Dictionary import Dictionary
from topologicpy.Topology import Topology
from topologicpy.Vertex import Vertex
from topologicpy.Graph import Graph
from topologicpy.Plotly import Plotly
import itertools
import dgl
import torch
import random

import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
from dgl import save_graphs, load_graphs
from tqdm.notebook import tqdm

/home/bruno_perdigao/repos/Plan-Classification-Topologic-DGL/.venv/lib/python3.12/site-packages/torchdata/datapipes/__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()


In [3]:
import dgl

In [4]:
pwd

'/home/bruno_perdigao/repos/Plan-Classification-Topologic-DGL'

In [5]:
path = "./data3"
dataset = DGL.DatasetByCSVPath(path, numberOfGraphClasses=0,
                           nodeATTRKey='feat', edgeATTRKey='feat',
                           nodeOneHotEncode=False, nodeFeaturesCategories=[],
                           edgeOneHotEncode=False, edgeFeaturesCategories=[], addSelfLoop=False)
print(dataset)

Done saving data into cached files.
Dataset("GraphDGL", num_graphs=200, save_path=/home/bruno_perdigao/.dgl/GraphDGL)


In [6]:
train_data, val_data, test_data = dgl.data.utils.split_dataset(dataset, [0.7, 0.1, 0.2])

In [7]:
train_data.dataset

Dataset("GraphDGL", num_graphs=200, save_path=/home/bruno_perdigao/.dgl/GraphDGL)

---

In [33]:
# Contruct a two-layer GNN model
# import dgl.nn as dglnn
# import torch.nn as nn
# import torch.nn.functional as F
# class SAGE(nn.Module):
#     def __init__(self, in_feats, hid_feats, out_feats):
#         super().__init__()
#         self.conv1 = dglnn.SAGEConv(
#             in_feats=in_feats, out_feats=hid_feats, aggregator_type='mean')
#         self.conv2 = dglnn.SAGEConv(
#             in_feats=hid_feats, out_feats=out_feats, aggregator_type='mean')

#     def forward(self, graph, inputs):
#         # inputs are features of nodes
#         h = self.conv1(graph, inputs)
#         h = F.relu(h)
#         h = self.conv2(graph, h)
#         return h

In [8]:
def evaluate(model, graph, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(graph, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [35]:
# # print(len(dataset.graphs))
# for i, graph in enumerate(dataset.graphs):
#     # print(i)
#     node_features = graph.ndata['feat']
#     # print(node_features)
#     node_labels = graph.ndata['label']
#     # print(node_labels)
#     train_mask = graph.ndata['train_mask']
#     valid_mask = graph.ndata['val_mask']
#     test_mask = graph.ndata['test_mask']
#     n_features = node_features.shape[1]
#     # print(n_features)
#     n_labels = int(node_labels.max().item() + 1)
#     # print(n_features)

# Grid Search

In [ ]:
# num_hidden_layers = [2, 3, 4, 5]
# num_units_per_layer = [25, 50, 100]
# activation_functions = [F.relu, F.tanh]
# learning_rates = [0.01, 0.001, 0.0005]
# batch_sizes = [16, 32, 64]
# aggregator_type = ['mean', 'pool', 'lstm']

In [9]:
def evaluate_selected_model(model):
    dictionary = DGL.ModelClassifyNodes(model, test_data.dataset)
    
    actual = []
    for l in dictionary['alllabels']:
        actual.extend(l)
        
    predicted = []
    for l in dictionary['allpredictions']:
        predicted.extend(l)

    result = DGL.Accuracy(actual=actual, predicted=predicted)
    accuracy = result['accuracy']
    precision = DGL.Precision(actual=actual, predicted=predicted)
    recall = DGL.Recall(actual=actual, predicted=predicted)
   
    return accuracy, precision, recall

class SAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, num_layers, activation, agg_type):
        super().__init__()
        self.layers = nn.ModuleList()
        self.activation = activation
        
        # Create the specified number of hidden layers
        self.layers.append(dglnn.SAGEConv(in_feats=in_feats, out_feats=hid_feats, aggregator_type=agg_type))
        for _ in range(num_layers - 1):
            self.layers.append(dglnn.SAGEConv(in_feats=hid_feats, out_feats=hid_feats, aggregator_type=agg_type))
        self.layers.append(dglnn.SAGEConv(in_feats=hid_feats, out_feats=out_feats, aggregator_type=agg_type))

    def forward(self, graph, inputs):
        h = inputs
        for layer in self.layers[:-1]:
            h = layer(graph, h)
            h = self.activation(h)  # Apply activation function
        h = self.layers[-1](graph, h)  # Last layer without activation
        return h


def train_with_grid(param_grid):
    results = []
    
    # Iterate over all combinations of hyperparameters
    for num_layers, hid_feats, activation, lr, batch_size, agg_type in tqdm(param_grid):
        print(f"Training with layers: {num_layers}, units per layer: {hid_feats}, activation: {activation.__name__}, learning rate: {lr}, batch size: {batch_size}, aggreagator type: {agg_type}")
        
        # Initialize model and optimizer
        n_features = 8
        model = SAGE(in_feats=n_features, hid_feats=hid_feats, out_feats=13, num_layers=num_layers, activation=activation, agg_type=agg_type)
        opt = torch.optim.Adam(model.parameters(), lr=lr)
        
        
    
        # for i, graph in enumerate(dataset.graphs):
        all_graph = dgl.batch(dataset.graphs)
            
        df = pd.DataFrame(columns=['epoch', 'loss', 'val_acc', 'time'])
        for epoch in range(100):
            start = time.time()        
            node_features = all_graph.ndata['feat']
            node_labels = all_graph.ndata['label']
            train_mask = all_graph.ndata['train_mask']
            valid_mask = all_graph.ndata['val_mask']
            test_mask = all_graph.ndata['test_mask']
            n_features = node_features.shape[1]
            n_labels = int(node_labels.max().item() + 1)

            model.train()
            # forward propagation by using all nodes
            logits = model(all_graph, node_features)
            # compute loss
            loss = F.cross_entropy(logits[train_mask], node_labels[train_mask])
            # compute validation accuracy
            acc = evaluate(model, all_graph, node_features, node_labels, valid_mask)
            # backward propagation
            opt.zero_grad()
            loss.backward()
            opt.step()
            total_time = time.time() - start
            df.loc[len(df)]={"epoch": epoch, "loss": loss.item(), "val_acc": acc, "time": total_time}
            # print(loss.item())

        accuracy, precision, recall = evaluate_selected_model(model)  
        result = {
            'num_hidden_layers': num_layers,
            'num_units_per_layer': hid_feats,
            'activation_functions': activation,
            'learning_rates': lr, 
            'batch_sizes': batch_size,
            'aggregator_type': agg_type,
            'acc': accuracy,
            'precision': precision,
            'recall': recall,
            'details': df,
        }
        results.append(result)
    return results, model

In [41]:
num_hidden_layers = [2, 3, 4, 5]
num_units_per_layer = [50]
activation_functions = [F.relu]
learning_rates = [0.001]
batch_sizes = [32]
aggregator_type = ['mean']

# Create a grid of hyperparameters
param_grid = list(itertools.product(num_hidden_layers, num_units_per_layer, activation_functions, learning_rates, batch_sizes, aggregator_type))

hidden_layers_results = train_with_grid(param_grid)
hidden_layers_results

  0%|          | 0/4 [00:00<?, ?it/s]

Training with layers: 2, units per layer: 50, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: mean
Training with layers: 3, units per layer: 50, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: mean
Training with layers: 4, units per layer: 50, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: mean
Training with layers: 5, units per layer: 50, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: mean


([{'num_hidden_layers': 2,
   'num_units_per_layer': 50,
   'activation_functions': <function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
   'learning_rates': 0.001,
   'batch_sizes': 32,
   'aggregator_type': 'mean',
   'acc': 0.7959,
   'precision': 0.7959,
   'recall': 0.7959,
   'details':     epoch       loss   val_acc      time
   0       0  15.905062  0.041005  0.017063
   1       1  11.314794  0.175926  0.014814
   2       2   8.912500  0.369048  0.024174
   3       3   7.652423  0.379630  0.022931
   4       4   6.080705  0.382275  0.017010
   ..    ...        ...       ...       ...
   95     95   0.698524  0.777778  0.015410
   96     96   0.695090  0.776455  0.015021
   97     97   0.691905  0.779101  0.011401
   98     98   0.688601  0.783069  0.017760
   99     99   0.685116  0.789683  0.014525
   
   [100 rows x 4 columns]},
  {'num_hidden_layers': 3,
   'num_units_per_layer': 50,
   'activation_functions': <function torch.nn.fu

In [31]:
num_hidden_layers = [3]
num_units_per_layer = [25, 50, 75, 100]
activation_functions = [F.relu]
learning_rates = [0.001]
batch_sizes = [32]
aggregator_type = ['mean']

# Create a grid of hyperparameters
param_grid = list(itertools.product(num_hidden_layers, num_units_per_layer, activation_functions, learning_rates, batch_sizes, aggregator_type))

num_units_results = train_with_grid(param_grid)
num_units_results

  0%|          | 0/4 [00:00<?, ?it/s]

Training with layers: 3, units per layer: 25, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: mean
Training with layers: 3, units per layer: 50, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: mean
Training with layers: 3, units per layer: 75, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: mean
Training with layers: 3, units per layer: 100, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: mean


([{'num_hidden_layers': 3,
   'num_units_per_layer': 25,
   'activation_functions': <function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
   'learning_rates': 0.001,
   'batch_sizes': 32,
   'aggregator_type': 'mean',
   'acc': 0.800507,
   'precision': 0.800507,
   'recall': 0.800507,
   'details':     epoch       loss   val_acc      time
   0       0  16.531101  0.037037  0.027119
   1       1  14.095226  0.144180  0.026290
   2       2  12.199215  0.187831  0.018760
   3       3  11.253201  0.292328  0.014049
   4       4  10.635205  0.309524  0.010842
   ..    ...        ...       ...       ...
   95     95   0.814908  0.796296  0.011889
   96     96   0.808954  0.800265  0.011298
   97     97   0.802853  0.802910  0.011160
   98     98   0.796753  0.802910  0.011314
   99     99   0.790802  0.802910  0.007514
   
   [100 rows x 4 columns]},
  {'num_hidden_layers': 3,
   'num_units_per_layer': 50,
   'activation_functions': <function torch

In [34]:
num_hidden_layers = [3]
num_units_per_layer = [50]
activation_functions = [F.relu, F.tanh]
learning_rates = [0.001]
batch_sizes = [32]
aggregator_type = ['mean']

# Create a grid of hyperparameters
param_grid = list(itertools.product(num_hidden_layers, num_units_per_layer, activation_functions, learning_rates, batch_sizes, aggregator_type))

activation_results = train_with_grid(param_grid)
activation_results

  0%|          | 0/2 [00:00<?, ?it/s]

Training with layers: 3, units per layer: 50, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: mean
Training with layers: 3, units per layer: 50, activation: tanh, learning rate: 0.001, batch size: 32, aggreagator type: mean


([{'num_hidden_layers': 3,
   'num_units_per_layer': 50,
   'activation_functions': <function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
   'learning_rates': 0.001,
   'batch_sizes': 32,
   'aggregator_type': 'mean',
   'acc': 0.828035,
   'precision': 0.828035,
   'recall': 0.828035,
   'details':     epoch       loss   val_acc      time
   0       0  29.167978  0.006614  0.034554
   1       1  20.938929  0.115079  0.022680
   2       2  17.364372  0.361111  0.023863
   3       3  15.249525  0.361111  0.021429
   4       4  12.758952  0.361111  0.017231
   ..    ...        ...       ...       ...
   95     95   0.702780  0.818783  0.026884
   96     96   0.696395  0.820106  0.018938
   97     97   0.692222  0.825397  0.019877
   98     98   0.687258  0.826720  0.019253
   99     99   0.681391  0.826720  0.019076
   
   [100 rows x 4 columns]},
  {'num_hidden_layers': 3,
   'num_units_per_layer': 50,
   'activation_functions': <function torch

In [33]:
num_hidden_layers = [3]
num_units_per_layer = [50]
activation_functions = [F.relu]
learning_rates = [0.01, 0.005, 0.001, 0.0005]
batch_sizes = [32]
aggregator_type = ['mean']

# Create a grid of hyperparameters
param_grid = list(itertools.product(num_hidden_layers, num_units_per_layer, activation_functions, learning_rates, batch_sizes, aggregator_type))

lr_results = train_with_grid(param_grid)
lr_results

  0%|          | 0/4 [00:00<?, ?it/s]

Training with layers: 3, units per layer: 50, activation: relu, learning rate: 0.01, batch size: 32, aggreagator type: mean
Training with layers: 3, units per layer: 50, activation: relu, learning rate: 0.005, batch size: 32, aggreagator type: mean
Training with layers: 3, units per layer: 50, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: mean
Training with layers: 3, units per layer: 50, activation: relu, learning rate: 0.0005, batch size: 32, aggreagator type: mean


([{'num_hidden_layers': 3,
   'num_units_per_layer': 50,
   'activation_functions': <function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
   'learning_rates': 0.01,
   'batch_sizes': 32,
   'aggregator_type': 'mean',
   'acc': 0.938148,
   'precision': 0.938148,
   'recall': 0.938148,
   'details':     epoch       loss   val_acc      time
   0       0  33.317741  0.025132  0.032303
   1       1  10.015996  0.436508  0.028816
   2       2  10.676816  0.339947  0.020405
   3       3   6.615351  0.354497  0.015750
   4       4   6.717198  0.382275  0.021194
   ..    ...        ...       ...       ...
   95     95   0.192654  0.937831  0.026063
   96     96   0.189751  0.939153  0.025700
   97     97   0.187232  0.936508  0.023719
   98     98   0.184835  0.937831  0.024848
   99     99   0.182431  0.937831  0.025036
   
   [100 rows x 4 columns]},
  {'num_hidden_layers': 3,
   'num_units_per_layer': 50,
   'activation_functions': <function torch.

In [35]:
num_hidden_layers = [3]
num_units_per_layer = [50]
activation_functions = [F.relu]
learning_rates = [0.001]
batch_sizes = [16, 32, 64]
aggregator_type = ['mean']

# Create a grid of hyperparameters
param_grid = list(itertools.product(num_hidden_layers, num_units_per_layer, activation_functions, learning_rates, batch_sizes, aggregator_type))

batch_results = train_with_grid(param_grid)
batch_results

  0%|          | 0/3 [00:00<?, ?it/s]

Training with layers: 3, units per layer: 50, activation: relu, learning rate: 0.001, batch size: 16, aggreagator type: mean
Training with layers: 3, units per layer: 50, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: mean
Training with layers: 3, units per layer: 50, activation: relu, learning rate: 0.001, batch size: 64, aggreagator type: mean


([{'num_hidden_layers': 3,
   'num_units_per_layer': 50,
   'activation_functions': <function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
   'learning_rates': 0.001,
   'batch_sizes': 16,
   'aggregator_type': 'mean',
   'acc': 0.739346,
   'precision': 0.739346,
   'recall': 0.739346,
   'details':     epoch       loss   val_acc      time
   0       0  87.209923  0.026455  0.035996
   1       1  78.510414  0.035714  0.032505
   2       2  71.699860  0.035714  0.028691
   3       3  64.395454  0.038360  0.024818
   4       4  56.775005  0.041005  0.022738
   ..    ...        ...       ...       ...
   95     95   0.827843  0.722222  0.018666
   96     96   0.820504  0.730159  0.019255
   97     97   0.812420  0.732804  0.016676
   98     98   0.808216  0.720899  0.017472
   99     99   0.804254  0.720899  0.020113
   
   [100 rows x 4 columns]},
  {'num_hidden_layers': 3,
   'num_units_per_layer': 50,
   'activation_functions': <function torch

In [36]:
num_hidden_layers = [3]
num_units_per_layer = [50]
activation_functions = [F.relu]
learning_rates = [0.001]
batch_sizes = [32]
aggregator_type = ['mean', 'pool', 'lstm']

# Create a grid of hyperparameters
param_grid = list(itertools.product(num_hidden_layers, num_units_per_layer, activation_functions, learning_rates, batch_sizes, aggregator_type))

agg_results = train_with_grid(param_grid)
agg_results

  0%|          | 0/3 [00:00<?, ?it/s]

Training with layers: 3, units per layer: 50, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: mean
Training with layers: 3, units per layer: 50, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: pool
Training with layers: 3, units per layer: 50, activation: relu, learning rate: 0.001, batch size: 32, aggreagator type: lstm


([{'num_hidden_layers': 3,
   'num_units_per_layer': 50,
   'activation_functions': <function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
   'learning_rates': 0.001,
   'batch_sizes': 32,
   'aggregator_type': 'mean',
   'acc': 0.826192,
   'precision': 0.826192,
   'recall': 0.826192,
   'details':     epoch       loss   val_acc      time
   0       0  31.819288  0.021164  0.022009
   1       1  25.051481  0.030423  0.016078
   2       2  18.280645  0.041005  0.014560
   3       3  12.709352  0.165344  0.013626
   4       4  10.021749  0.298942  0.017126
   ..    ...        ...       ...       ...
   95     95   0.573344  0.810847  0.020077
   96     96   0.569976  0.817460  0.017157
   97     97   0.566920  0.816138  0.015732
   98     98   0.563653  0.820106  0.017582
   99     99   0.560235  0.826720  0.016262
   
   [100 rows x 4 columns]},
  {'num_hidden_layers': 3,
   'num_units_per_layer': 50,
   'activation_functions': <function torch

In [38]:
num_hidden_layers = [4]
num_units_per_layer = [100]
activation_functions = [F.relu]
learning_rates = [0.01]
batch_sizes = [64]
aggregator_type = ['lstm']

# Create a grid of hyperparameters
param_grid = list(itertools.product(num_hidden_layers, num_units_per_layer, activation_functions, learning_rates, batch_sizes, aggregator_type))

fine_results, model = train_with_grid(param_grid)
fine_results

  0%|          | 0/1 [00:00<?, ?it/s]

Training with layers: 4, units per layer: 100, activation: relu, learning rate: 0.01, batch size: 64, aggreagator type: lstm


[{'num_hidden_layers': 4,
  'num_units_per_layer': 100,
  'activation_functions': <function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
  'learning_rates': 0.01,
  'batch_sizes': 64,
  'aggregator_type': 'lstm',
  'acc': 0.955425,
  'precision': 0.955425,
  'recall': 0.955425,
  'details':     epoch      loss   val_acc      time
  0       0  6.476654  0.107143  0.574064
  1       1  5.111220  0.653439  0.438891
  2       2  5.511761  0.546296  0.511938
  3       3  5.938691  0.361111  0.660172
  4       4  4.010347  0.726190  0.681041
  ..    ...       ...       ...       ...
  95     95  0.125232  0.955026  0.689732
  96     96  0.125232  0.960317  0.709354
  97     97  0.120529  0.957672  0.651969
  98     98  0.118559  0.958995  0.874100
  99     99  0.119317  0.962963  0.598034
  
  [100 rows x 4 columns]}]

In [ ]:
fine_results

In [42]:
dictionary = DGL.ModelClassifyNodes(model, test_data.dataset)
# actual = dictionary['alllabels']
actual = []
for l in dictionary['alllabels']:
    actual.extend(l)
    
# print(actual)
# predicted = dictionary['allpredictions']
predicted = []
for l in dictionary['allpredictions']:
    predicted.extend(l)

In [43]:
cf = DGL.ConfusionMatrix(actual = actual, predicted = predicted)
print(cf)
fig = Plotly.FigureByConfusionMatrix(cf)
Plotly.Show(fig)

[[ 160    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0  120    0    0    0    0    0    0    0    9    0    0    0]
 [   0    0  146    0    0   27    0    0    0    0    0    0    0]
 [   0    0   17   87    0   48    0    0    0    0    0    0    0]
 [   0    0    0    0  177    0   23    7    1    0    0    0    0]
 [   0    0   77  113    0  407    0    0    0    0    0    0    0]
 [   0    0    0    0   10    0  408   27    0    0    0    0    0]
 [   0    0    0    0   11    0   13  166    0    0    0    0    0]
 [   0    0    0    0    2    0    0    0  139    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0  231    0    0    0]
 [   0    0    0    0    0    0    0    0    2    0  222    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0 3263    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0 2769]]


In [44]:
categories = set(actual+predicted)
categories
true_positives = {category: 0 for category in categories}
false_positives = {category: 0 for category in categories}
false_negatives = {category: 0 for category in categories}
n_labels = {category: 0 for category in categories}


for i in range(len(predicted)):
    if predicted[i] == actual[i]:
        true_positives[actual[i]] += 1
    else:
        false_positives[predicted[i]] += 1

for i in range(len(predicted)):
    if predicted[i] == actual[i]:
        pass
    else:
        false_negatives[actual[i]] += 1

for i in range(len(categories)):
    n_labels[i] = true_positives[i] + false_negatives[i]

true_positives
false_positives

total_true_positives = sum(true_positives.values())
total_false_positives = sum(false_positives.values())
total_true_positives / (total_true_positives + total_false_positives)

0.955425017277125

In [45]:
for c in range(len(categories)):
    if (true_positives[c] + false_positives[c]) == 0:
        precision = 0
    else:
        precision = true_positives[c] / (true_positives[c] + false_positives[c])

    if (true_positives[c] + false_negatives[c]) == 0:
        recall = 0
    else:
        recall = true_positives[c] / (true_positives[c] + false_negatives[c])
    # if (true_positives[c] + n_labels[c]) == 0:
    #     accuracy = 0
    # else:
    #     accuracy = true_positives[c]/ (n_labels[c])
    
    # print(f"accuracy for {c}: {accuracy}")
    print(c)
    print(f"precision for {c}: {precision}")
    print(f"recall for {c}: {recall}")
    print("---")

0
precision for 0: 1.0
recall for 0: 1.0
---
1
precision for 1: 0.9302325581395349
recall for 1: 1.0
---
2
precision for 2: 0.8439306358381503
recall for 2: 0.6083333333333333
---
3
precision for 3: 0.5723684210526315
recall for 3: 0.435
---
4
precision for 4: 0.8509615384615384
recall for 4: 0.885
---
5
precision for 5: 0.6817420435510888
recall for 5: 0.8443983402489627
---
6
precision for 6: 0.9168539325842696
recall for 6: 0.918918918918919
---
7
precision for 7: 0.8736842105263158
recall for 7: 0.83
---
8
precision for 8: 0.9858156028368794
recall for 8: 0.9788732394366197
---
9
precision for 9: 1.0
recall for 9: 0.9625
---
10
precision for 10: 0.9910714285714286
recall for 10: 1.0
---
11
precision for 11: 1.0
recall for 11: 1.0
---
12
precision for 12: 1.0
recall for 12: 1.0
---
